# Zadania - klasyfikacja wieloklasowa

Celem zadań jest zbudowanie klasyfikatora rozpoznającego ręcznie zapisaną cyfrę 0-9 z pojedynczego czarno-białego obrazu. Zestaw danych, na których będziemy pracować to MNIST (https://en.wikipedia.org/wiki/MNIST_database).

MNIST zawiera ok. 70000 obrazów, każdy oznaczony cyfrą, którą przedstawia. Pojedyncze obrazy w tym zbiorze danych wyglądają np. tak: 

<img src="http://pavel.surmenok.com/wp-content/uploads/2014/07/mnistdigits.gif"/>

Zbiór jest już podzielony na zestawy treningowy i testowy. 

## MNIST

Pobierz cztery pliki `*-images-idx3-ubyte.gz` ze strony http://yann.lecun.com/exdb/mnist/ i rozpakuj (!) w tym samym katalogu, w którym znajduje się notatnik (pod Linuxem możesz wykorzystać skrypt `download.sh`). **Uwaga**: proszę nie wrzucać tych plików do swojego repozytorium.

1. Wyświetl w postaci tablicy oraz w postaci obrazka pierwsze pięć liczb. Wykorzystaj do tego funkcje `readMnist` i `showImage` (nie jest potrzebna znajomość ich działania).
1. Stwórz macierze dla zestawów treningowego i testowego, zawierające pierwsze 1000 obrazków każdy (wykorzystaj `maxItems` w funkcji `mnistMatrix`, podanie `maxItems=60000` wczyta wszystkie dane). Odpowiedz na pytania: ile cech ma każdy obrazek? Czym są te cechy? Jak przedstawione są etykiety w zbiorze testowym?

## Regresja logistyczna

1. Na podstawie materiałów z wykładu nr 5 zbuduj model wieloklasowej regresji logistycznej rozpoznający wszystkie 10 cyfr na podstawie pierwszych 1000 danych. Eksperymentuj z doborem parametrów w algorytmie GD. Uwaga: należy rozpocząć od małej liczby kroków, np. 100, i powoli zwiększać. Jaką osiągnięto poprawność (_accuracy_)?
1. Zwiększ liczbę danych treningowych do 10000, 20000 itd. Co się dzieje z algorytmem GD?
1. Zaimplementuj algorytm Batch-SGD (_Batch Stochastic Gradient Descent_), którego wykrzystasz w miejsce podstawowego algorytmu. Różnice między tymi algorytmami obrazują poniższe pseudokody:

   Gradient Descent
   ```
   for i in range(num_epochs):
       gradient = evaluate_gradient(J, data, thetas)
       thetas = thetas - alpha * gradient
   ```
   
   Batch Stochastic Gradient Descent
   ```
   for i in range(num_epochs):
       shuffle(data)
       for batch in get_batch(data, batch_size=50):
           gradient = evaluate_gradient(J, batch, thetas)
           thetas = thetas - alpha * gradient
    ```
    
    Gdzie `num_epochs` to liczba epok, a epoką nazywamy jednokrotne wykorzystanie wszystkich danych w zbiorze treningowym do nauki parametrów (do tej pory zakładaliśmy `num_epoch=1`).

1. Wykorzystaj Batch-SGD do zbudowania modelu na całosci danych treningowych (można zastosować jedną epokę i pominąć randomizację dla uzyskania stabilnych wyników). Jaką osiągnięto poprawność, dopasowując parametry `alpha`, `maxSteps` i `batchSize`? Zmiana których parametrów ma najmniejszy/największy wpływ na osiągany wynik? Które parametry są łatwe/trudne do dopasowywania?

In [1]:
import os
import struct
import numpy as np

%matplotlib inline

def readMnist(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

def showImage(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

In [2]:
def mnistMatrix(data, maxItems=1000):
    datalist = [t for t in data]
    m = maxItems
    n = 28 * 28 + 1
    X = np.matrix(np.zeros(m * n)).reshape(m, n)
    Y = np.matrix(np.zeros(m)).reshape(m, 1)
    for i, (label, image) in enumerate(datalist[:m]):
        X[i, 0] = 1 # bias term
        X[i, 1:] = image.reshape(28*28,)
        Y[i] = label
    return X, Y